In [ ]:
!pip install catboost

     |████████████████████████████████| 66.2MB 61kB/s 


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

Dataset link: https://datahack.analyticsvidhya.com/contest/practice-problem-loan-prediction-iii/

In [ ]:
#reading the dataset
df=pd.read_csv("train.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [ ]:
df.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [ ]:
cols = df.columns

for i in cols:
    if df[i].isnull().sum() != 0:
        print("Column name is:", i)
        print(df[i].isnull().sum())

Column name is: Gender
13
Column name is: Married
3
Column name is: Dependents
15
Column name is: Self_Employed
32
Column name is: LoanAmount
22
Column name is: Loan_Amount_Term
14
Column name is: Credit_History
50


In [ ]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [ ]:
#filling missing values
print(df['Gender'].value_counts())
df['Gender'].fillna('Male', inplace=True)

print(df['Married'].value_counts())
df['Married'].fillna('Yes', inplace=True)

print(df['Dependents'].value_counts())
df['Dependents'].fillna('0', inplace=True)

print(df['Self_Employed'].value_counts())
df['Self_Employed'].fillna('No', inplace=True)

print(df.LoanAmount.describe())
df['LoanAmount'].fillna(df.LoanAmount.mean(), inplace = True)

print(df['Loan_Amount_Term'].value_counts())
df['Loan_Amount_Term'].fillna(512, inplace=True)

print(df['Credit_History'].value_counts())
df['Credit_History'].fillna(1.0, inplace=True)

Male      489
Female    112
Name: Gender, dtype: int64
Yes    398
No     213
Name: Married, dtype: int64
0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64
No     500
Yes     82
Name: Self_Employed, dtype: int64
count    592.000000
mean     146.412162
std       85.587325
min        9.000000
25%      100.000000
50%      128.000000
75%      168.000000
max      700.000000
Name: LoanAmount, dtype: float64
360.0    512
180.0     44
480.0     15
300.0     13
84.0       4
240.0      4
120.0      3
36.0       2
60.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64
1.0    475
0.0     89
Name: Credit_History, dtype: int64


In [ ]:
# Get categorical columns
cat_cols = []
for i in cols:
    if df[i].dtypes == 'object' and i!= 'Loan_ID':
        print(i)
        cat_cols.append(i)

# Do label encoding for categorical columns
le = LabelEncoder()
for i in cat_cols:
    df[i] = le.fit_transform(df[i])

Gender
Married
Dependents
Education
Self_Employed
Property_Area
Loan_Status


In [ ]:
#split dataset into train and test

train, test = train_test_split(df, test_size=0.3, random_state=0)

x_train=train.drop(['Loan_Status', 'Loan_ID'], axis=1)
y_train=train['Loan_Status']

x_test=test.drop(['Loan_Status', 'Loan_ID'], axis=1)
y_test=test['Loan_Status']

#### LogisticRegression 

In [ ]:
model = LogisticRegression(random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8324324324324325

#### NaiveBayes 

In [ ]:
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.8216216216216217

#### SVM 

In [ ]:
model = svm.SVC()
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.7297297297297297

#### Decission Tree 

In [ ]:
model = tree.DecisionTreeClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.7621621621621621

#### Bagging Classifier 

In [ ]:
model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.7189189189189189

#### Random Forest 

In [ ]:
model= RandomForestClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.7945945945945946

In [ ]:
# Get feature importance
for i, j in sorted(zip(x_train.columns, model.feature_importances_)):
    print(i, j)

ApplicantIncome 0.19748399810004863
CoapplicantIncome 0.11872820931486824
Credit_History 0.2613040128071408
Dependents 0.05134989642175094
Education 0.024437887453173667
Gender 0.020308953705222458
LoanAmount 0.18829346004307884
Loan_Amount_Term 0.03561738164859797
Married 0.025545664769708782
Property_Area 0.05676534088493939
Self_Employed 0.020165194851470426


#### Adaboost 

In [ ]:
model = AdaBoostClassifier(random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.8162162162162162

#### Gradient boosting classifier 

In [ ]:
model= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.827027027027027

#### XGBoost 

In [ ]:
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(x_train, y_train)
model.score(x_test,y_test)

0.827027027027027

#### LightGBM 

In [ ]:
train_data=lgb.Dataset(x_train,label=y_train)
#define parameters
params = {'learning_rate':0.001}
model= lgb.train(params, train_data, 100) 
y_pred=model.predict(x_test)
for i in range(0,185):
    if y_pred[i]>=0.5: 
        y_pred[i]=1
else: 
    y_pred[i]=0
accuracy_score(y_test, y_pred)

0.7189189189189189

#### CatBoost 

In [ ]:
model=CatBoostClassifier()
categorical_features_indices = np.where(df.dtypes != np.float)[0]
model.fit(x_train,y_train,cat_features=([ 0,  1, 2, 3, 4, 10]),eval_set=(x_test, y_test))
model.score(x_test,y_test)

Learning rate set to 0.025711
0:	learn: 0.6794857	test: 0.6808164	best: 0.6808164 (0)	total: 53.3ms	remaining: 53.3s
1:	learn: 0.6686164	test: 0.6694985	best: 0.6694985 (1)	total: 54.7ms	remaining: 27.3s
2:	learn: 0.6580962	test: 0.6584226	best: 0.6584226 (2)	total: 55.9ms	remaining: 18.6s
3:	learn: 0.6466061	test: 0.6481062	best: 0.6481062 (3)	total: 58.5ms	remaining: 14.6s
4:	learn: 0.6361742	test: 0.6382075	best: 0.6382075 (4)	total: 61ms	remaining: 12.1s
5:	learn: 0.6267725	test: 0.6280762	best: 0.6280762 (5)	total: 62.5ms	remaining: 10.4s
6:	learn: 0.6176542	test: 0.6189313	best: 0.6189313 (6)	total: 65ms	remaining: 9.21s
7:	learn: 0.6088622	test: 0.6105600	best: 0.6105600 (7)	total: 67.4ms	remaining: 8.36s
8:	learn: 0.5994512	test: 0.6021100	best: 0.6021100 (8)	total: 69.8ms	remaining: 7.68s
9:	learn: 0.5926521	test: 0.5948715	best: 0.5948715 (9)	total: 70.7ms	remaining: 7s
10:	learn: 0.5846648	test: 0.5867226	best: 0.5867226 (10)	total: 73.1ms	remaining: 6.57s
11:	learn: 0.57719

0.8162162162162162

#### MaxVoting 

In [ ]:
model1 = LogisticRegression(random_state=1)
model2 = tree.DecisionTreeClassifier(random_state=1)
model3 = GaussianNB()
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2), ('NB', model3)])
model.fit(x_train,y_train)
model.score(x_test,y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.827027027027027

#### Weighted Averaging 

In [ ]:
model1 = tree.DecisionTreeClassifier()
model2 = GaussianNB()
model3= LogisticRegression()


model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)

pred1 = model1.predict_proba(x_test)
pred2 = model2.predict_proba(x_test)
pred3 = model3.predict_proba(x_test)

weighted_prediction = (0.2*pred1)+(0.4*pred2)+(0.4*pred3)
labelprediction = np.argmax(weighted_prediction, axis = 1)

accuracy_score(labelprediction, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.827027027027027

#### Stacking 

In [ ]:
def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    test_pred=np.empty((test.shape[0],1),float)
    train_pred=np.empty((0,1),float)
    
    for train_indices,val_indices in folds.split(train,y.values):
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
        
    model.fit(train, y)
    test_pred=model.predict(test)
    return test_pred.reshape(-1,1),train_pred

In [ ]:
model1 = tree.DecisionTreeClassifier(random_state=1)

test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10, train=x_train,test=x_test,y=y_train)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
model2 = LogisticRegression()

test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=x_train,test=x_test,y=y_train)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the n

In [ ]:
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,y_train)
model.score(df_test, y_test)

0.8324324324324325

#### Blending 

In [ ]:
train, test = train_test_split(train, test_size=0.2, random_state=0)

x_train=train.drop(['Loan_Status', 'Loan_ID'], axis=1)
y_train=train['Loan_Status']

x_val=test.drop(['Loan_Status', 'Loan_ID'], axis=1)
y_val=test['Loan_Status']

x_val = x_val.reset_index(drop = True)
x_test = x_test.reset_index(drop = True)

model1 = tree.DecisionTreeClassifier()
model1.fit(x_train, y_train)

val_pred1=model1.predict(x_val)
test_pred1=model1.predict(x_test)

val_pred1=pd.DataFrame(val_pred1)
test_pred1=pd.DataFrame(test_pred1)

model2 = LogisticRegression()
model2.fit(x_train,y_train)

val_pred2=model2.predict(x_val)
test_pred2=model2.predict(x_test)

val_pred2=pd.DataFrame(val_pred2)
test_pred2=pd.DataFrame(test_pred2)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
df_val = pd.concat([x_val, val_pred1,val_pred2],axis=1)
df_test = pd.concat([x_test, test_pred1,test_pred2],axis=1)

model = LogisticRegression(random_state=1)
model.fit(df_val,y_val)
model.score(df_test,y_test)

0.8162162162162162

References:

    1. https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/
    2. https://mlwave.com/kaggle-ensembling-guide/
    3. https://www.analyticsvidhya.com/blog/2017/06/which-algorithm-takes-the-crown-light-gbm-vs-xgboost/
    4. https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc
    5. https://www.analyticsvidhya.com/blog/2017/08/catboost-automated-categorical-data/